In [1]:
%load_ext autoreload
%autoreload 2

In [98]:
# from main import get_normalized_probability, cal_most_prob_scene, reverse_relation
import json
import pulp
import numpy as np
from LPConstraints import get_clevr_block_world_constraint
from utils import process_coord_scene

In [109]:
with open('../results/clevr_block_val/constraint_1234/scenes_coord_prob.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('../data/clevr_block/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [110]:
scenes = [process_coord_scene(scene, 'blockworld') for scene in scenes]

In [111]:
scenes[0]

{'image_index': 0,
 'objects': [{'color': [0.10307889431715012,
    0.10457025468349457,
    0.10198843479156494,
    0.10117006301879883,
    0.10283872485160828,
    0.1016504168510437,
    0.28262683749198914,
    0.10207632929086685],
   'mask': {'size': [320, 480],
    'counts': 'hoP19e94J7L3L4M2N2N2N2N101N2O0O2O0O10000O1000000000000000000001O001O001N101N2O0O2N2N2N2M3N3L`XW3'},
   'shape': [0.5749544501304626, 0.21360108256340027, 0.21144454181194305],
   'material': [0.26887354254722595, 0.7311264276504517],
   'size': [0.7294363379478455, 0.2705637216567993]},
  {'color': [0.28117287158966064,
    0.1021944209933281,
    0.10317713022232056,
    0.1028904914855957,
    0.10222049057483673,
    0.10279274731874466,
    0.10296396166086197,
    0.10258792340755463],
   'mask': {'size': [320, 480],
    'counts': 'SgV38c99J4M2M4M3M1O3M101N2O1N1O2O00000O100000000000000001O0000001O1N100O2N2O0O2O1N2N3M3L3NSkQ1'},
   'shape': [0.5740546584129333, 0.2132999151945114, 0.21264544129371643]

In [112]:
def get_attribute_variables(objects, attributes_map):
    variables = []
    probabilities = {}
    num_objects = len(objects)

    for name, values in attributes_map.items():
        for v, _ in enumerate(values):
            for i in range(num_objects):
                variable_name = f'{name}_{i}_{v}'
                variables.append(variable_name)
                probabilities[variable_name] = objects[i][name][v]
    return variables, probabilities

def get_relationship_variables(scene, relationships):
    variables = []
    probabilities = {}

    for rel in relationships:
        for target, sources in enumerate(scene['relationships'][rel]):
            for source, probability in sources:
                # TODO: change the prediction for below and above (currently flipped)
                variable_name = f'{rel}_{source}_{target}'
                variables.append(variable_name)
                probabilities[variable_name] = probability
    return variables, probabilities

def get_LP_problem(scene, attr_map, eps=1e-50):
    attr_variables, attr_probabilities = get_attribute_variables(scene['objects'], attr_map['attributes'])
    rel_variables, rel_probabilities = get_relationship_variables(scene, attr_map['relations'])


    attr_variables = pulp.LpVariable.dict('attr', attr_variables, 0, 1, pulp.LpBinary)
    rel_variables = pulp.LpVariable.dict('rel', rel_variables, 0, 1, pulp.LpBinary)

    attr_objective = [attr_variables[i] * np.log(max(attr_probabilities[i], eps)) for i in attr_variables.keys()]
    rel_objective = [rel_variables[i] * np.log(max(rel_probabilities[i], eps)) + (1 - rel_variables[i]) * np.log((max(1 - rel_probabilities[i], eps))) for i in rel_variables.keys()]

    prob = pulp.LpProblem("sceneGraphProblem", pulp.LpMaximize)
    prob += pulp.lpSum(attr_objective + rel_objective)
    return prob, attr_variables, rel_variables

def get_predicted_scene_for_block_world(variables, scene):
    predicted_scene = {
        'objects': [{'mask': o['mask']} for o in scene['objects']],
        'relationships': { rel: [[] for _ in range(len(scene['objects']))] for rel in attr_map['relations']}
    }

    for v in variables:
        if v.varValue:
            tokens = v.name.split('_')
            if tokens[0] == 'attr':
                predicted_scene['objects'][int(tokens[2])][tokens[1]] = attr_map['attributes'][tokens[1]][int(tokens[3])]
            elif tokens[0] == 'rel':
                predicted_scene['relationships'][tokens[1]][int(tokens[3])].append(int(tokens[2]))
    return predicted_scene

In [113]:
from tqdm import tqdm
M = 100
predicted_scenes = []
solver = pulp.GUROBI_CMD(path = 'C:/gurobi951/win64/bin/gurobi_cl.exe')

for scene in tqdm(scenes):
    prob, attr_variables, rel_variables = get_LP_problem(scene, attr_map)
    get_clevr_block_world_constraint(prob, scene, rel_variables, attr_variables, attr_map, include_small_obj=True, include_large_cube=True, include_bottom_stack=True, include_yellow=True)
    prob.solve(solver)
    predicted_scenes.append(get_predicted_scene_for_block_world(prob.variables(), scene))

100%|██████████| 6000/6000 [10:03<00:00,  9.94it/s]


In [115]:
predicted_scenes[0]

{'objects': [{'mask': {'size': [320, 480],
    'counts': 'hoP19e94J7L3L4M2N2N2N2N101N2O0O2O0O10000O1000000000000000000001O001O001N101N2O0O2N2N2N2M3N3L`XW3'},
   'color': 'red',
   'material': 'rubber',
   'shape': 'sphere',
   'size': 'large'},
  {'mask': {'size': [320, 480],
    'counts': 'SgV38c99J4M2M4M3M1O3M101N2O1N1O2O00000O100000000000000001O0000001O1N100O2N2O0O2O1N2N3M3L3NSkQ1'},
   'color': 'blue',
   'material': 'rubber',
   'shape': 'sphere',
   'size': 'large'},
  {'mask': {'size': [320, 480],
    'counts': 'XbU2j0U91O2O0O1O101O0O100O100000000000001O0001O0000000O1000000O2O0O1O100O1Om_U2'},
   'color': 'cyan',
   'material': 'rubber',
   'shape': 'cube',
   'size': 'small'},
  {'mask': {'size': [320, 480],
    'counts': 'hTo0;d96Ih0ZO1O0O2N100O2N10000O1O2O000001O001O01O000001O001O000000000000O100O100000000O1O1000O01O01O100O1O100O1O2O0O1O2N102La]T3'},
   'color': 'gray',
   'material': 'rubber',
   'shape': 'cube',
   'size': 'large'},
  {'mask': {'size': [320, 480],
    'coun

In [114]:
with open('../results/clevr_block_val/constraint_1234/scene_prob_fix.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)



In [173]:
attr_variables

{'shape_0_0': attr_shape_0_0,
 'shape_1_0': attr_shape_1_0,
 'shape_2_0': attr_shape_2_0,
 'shape_0_1': attr_shape_0_1,
 'shape_1_1': attr_shape_1_1,
 'shape_2_1': attr_shape_2_1,
 'shape_0_2': attr_shape_0_2,
 'shape_1_2': attr_shape_1_2,
 'shape_2_2': attr_shape_2_2,
 'size_0_0': attr_size_0_0,
 'size_1_0': attr_size_1_0,
 'size_2_0': attr_size_2_0,
 'size_0_1': attr_size_0_1,
 'size_1_1': attr_size_1_1,
 'size_2_1': attr_size_2_1,
 'material_0_0': attr_material_0_0,
 'material_1_0': attr_material_1_0,
 'material_2_0': attr_material_2_0,
 'material_0_1': attr_material_0_1,
 'material_1_1': attr_material_1_1,
 'material_2_1': attr_material_2_1,
 'color_0_0': attr_color_0_0,
 'color_1_0': attr_color_1_0,
 'color_2_0': attr_color_2_0,
 'color_0_1': attr_color_0_1,
 'color_1_1': attr_color_1_1,
 'color_2_1': attr_color_2_1,
 'color_0_2': attr_color_0_2,
 'color_1_2': attr_color_1_2,
 'color_2_2': attr_color_2_2,
 'color_0_3': attr_color_0_3,
 'color_1_3': attr_color_1_3,
 'color_2_3': at

In [203]:
print(len(prob.constraints))

174


In [169]:
rel_variables[f'left_{3}_{5}'].varValue

1.0

In [154]:
scene_file = '../data/clevr_block/constraint_1_2_3_4/scenes.json'

with open(scene_file) as f:
    gt_scenes = json.load(f)

In [155]:
for scene in gt_scenes['scenes']:
    scene['relationships']['above'], scene['relationships']['below'] = scene['relationships']['below'], scene['relationships']['above']

In [156]:
with open(scene_file, 'w') as f:
    json.dump(gt_scenes, f)